In [ ]:
!nvidia-smi

In [ ]:
import os
import shutil
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
from google.colab import drive
from google.colab import userdata
from roboflow import Roboflow
from ultralytics import YOLO
import os

In [ ]:
HOME = os.getcwd()
print(HOME)
ultralytics.checks()

In [ ]:
drive.mount('/content/drive')

In [ ]:
save_path = "/content/drive/MyDrive/pothole_detection"
os.makedirs(save_path, exist_ok=True)

In [ ]:
%cd {HOME}

if os.path.exists("/content/datasets_classification"):
    shutil.rmtree("/content/datasets_classification")
os.makedirs("/content/datasets_classification", exist_ok=True)

%cd {HOME}/datasets_classification


rf = Roboflow(api_key="vJRRNV9HbDPFy95m0veX")
project = rf.workspace("object-detection-pxcal").project("pothole-dif-severity-2")
version = project.version(2)
dataset_classification = version.download("yolov11")

In [ ]:
if os.path.exists("/content/drive/MyDrive/pothole_detection/yolo11l_model"):
    shutil.rmtree("/content/drive/MyDrive/pothole_detection/yolo11l_model")

In [ ]:
#model = YOLO("yolo5m.pt")
#model = YOLO("yolo5l.pt")
#model = YOLO("yolo11m.pt")
model = YOLO("yolo11l.pt")

yolo_params = {
    'task': 'detect',
    'image_size': 640,
    'batch_size': 16,
    'epochs': 450,
}

output_dir = "/content/drive/MyDrive/yolo11_pothole_project"
dataset_yaml_path = os.path.join(dataset_classification.location, "data.yaml")

results = model.train(
    data=dataset_yaml_path,
    task=yolo_params['task'],
    imgsz=yolo_params['image_size'],
    epochs=yolo_params['epochs'],
    name=f"yolo11l_model",
    project=os.path.join(output_dir, "runs"),
)

In [ ]:
result_dir = '/content/drive/MyDrive/pothole_detection/yolo11l_model/'
if os.path.exists(f"{result_dir}/predictions"):
    shutil.rmtree(f"{result_dir}/predictions")

In [ ]:
PREDICTION_DIR = f"{result_dir}/predictions"
os.makedirs(PREDICTION_DIR, exist_ok=True)
model = YOLO(f"{result_dir}/weights/best.pt")
results = model.predict(
    source=f'{dataset_classification.location}/test/images',
    imgsz=640,
    conf=0.25,
    iou=0.45,
    save=True,
    save_txt=True,
    project=PREDICTION_DIR,
    name="test_results"
)

metrics = model.val(
    data=dataset_yaml_path,
    split='test',
    imgsz=640,
    batch=16,
    project=PREDICTION_DIR,
    name="metric"
)